In [1]:
import json
import os
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm

from torch import Tensor
from torch import optim
from torch.utils.data import Dataset, DataLoader

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

device

'cpu'

In [2]:
# Constants

MAX_LENGTH = 40
SOS_TOKEN = 69
EOS_TOKEN = 70
TEACHER_FORCING_RATIO = 0.5
LEARNING_RATE = 0.01
HIDDEN_SIZE = 256

In [3]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size: int, hidden_size: int) -> None:
        super().__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input: torch.Tensor, hidden: torch.Tensor) -> torch.Tensor:
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def init_hidden(self) -> torch.Tensor:
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH) -> None:
        super().__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
    
    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [4]:
def sentence_to_tensor(content, target_size = MAX_LENGTH) -> torch.Tensor:
    # Add padding to the end of the sentence, so that the length is equal to target_size
    tensor = torch.tensor(content, dtype=torch.long, device=device).view(-1, 1)

    if tensor.size()[0] < target_size:
        padding = torch.zeros(target_size - tensor.size()[0], 1, dtype=torch.int32, device=device)
        tensor = torch.cat((tensor, padding), dim=0)
        
    return tensor

In [27]:
class TranscriptionDataset(Dataset):
    def __init__(self, path_to_words: str = 'data/word-based/words.csv'):
        self.__words = pd.read_csv(path_to_words)

    def __len__(self):
        return self.__words.shape[0]

    def __getitem__(self, idx):
        if idx >= self.__len__():
            raise IndexError
        
        row = self.__words.iloc[idx]

        input_tensor = sentence_to_tensor(json.loads(row[2]))
        target_tensor = sentence_to_tensor(json.loads(row[3]))
        return input_tensor, target_tensor


In [32]:
class Trainer:
    def __init__(self, encoder: EncoderRNN, decoder: DecoderRNN,
                 encoder_optimizer: optim.Optimizer, decoder_optimizer: optim.Optimizer,
                 max_length: int = MAX_LENGTH):
        self.__encoder = encoder.to(device)
        self.__decoder = decoder.to(device)
        self.__encoder_optimizer = encoder_optimizer
        self.__decoder_optimizer = decoder_optimizer
        self.__max_length = max_length
        self.__loss = 0

    def __init_train(self):
        encoder_hidden = self.__encoder.init_hidden()

        self.__encoder_optimizer.zero_grad()
        self.__decoder_optimizer.zero_grad()

        encoder_outputs = torch.zeros(self.__max_length, self.__encoder.hidden_size, device=device)

        self.__loss = 0

        return encoder_hidden, encoder_outputs
    
    def __encoder_train(self, encoder_outputs, input_tensor, encoder_hidden):
        input_length = input_tensor.size(0)

        for ei in range(input_length):
            encoder_output, encoder_hidden = self.__encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        return encoder_outputs

    def __optimizers_step(self):
        self.__encoder_optimizer.step()
        self.__decoder_optimizer.step()

    def __decoder_train(self, decoder_input, decoder_hidden, encoder_outputs, target_tensor, criterion):
        target_length = target_tensor.size(0)
        use_teacher_forcing = True if random.random() < TEACHER_FORCING_RATIO else False

        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = self.__decoder(
                    decoder_input, decoder_hidden, encoder_outputs)

            if use_teacher_forcing:
                decoder_input = target_tensor[di]
            else:
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze().detach()
            
            self.__loss += criterion(decoder_output, target_tensor[di])

            if use_teacher_forcing and decoder_input.item() == EOS_TOKEN:
                break

    def train(self, input_tensor: Tensor, target_tensor: Tensor, 
              criterion: nn.Module, max_length: int = MAX_LENGTH) -> tuple[float, float]:        

        # Encoder training
        encoder_hidden, encoder_outputs = self.__init_train()
        encoder_outputs = self.__encoder_train(encoder_outputs, input_tensor, encoder_hidden)

        # Decoder training
        decoder_input = torch.tensor([[SOS_TOKEN]], device=device)
        decoder_hidden = encoder_hidden

        self.__decoder_train(decoder_input, decoder_hidden, encoder_outputs, target_tensor, criterion)
        
        # Optimizers step
        self.__loss.backward()
        self.__optimizers_step()

        return self.__loss.item() / target_tensor.size(0)

In [31]:
def train_loop(encoder: EncoderRNN, decoder: DecoderRNN, dataset: TranscriptionDataset,
               epochs: int, print_every: int = 100):
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=LEARNING_RATE)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=LEARNING_RATE)
    criterion = nn.NLLLoss()


    trainer = Trainer(encoder, decoder, encoder_optimizer, decoder_optimizer)
    
    for epoch in range(epochs):
        total_loss = 0

        # for iteration, (input_tensor, target_tensor) in tqdm.tqdm(enumerate(dataset)):
        for iteration, (input_tensor, target_tensor) in enumerate(dataset):
            print(f'Input tensor shape: {input_tensor.shape}')
            print(f'Target tensor shape: {target_tensor.shape}')
            loss = trainer.train(input_tensor, target_tensor, criterion)
            total_loss += loss

            if iteration % print_every == 0:
                print(f'Epoch: {epoch} Iteration: {iteration} loss: {loss}')
                print(f'Average loss: {total_loss / iteration}')

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [30]:
input_size = 50
output_size = 50

encoder = EncoderRNN(MAX_LENGTH, HIDDEN_SIZE).to(device)
decoder = DecoderRNN(HIDDEN_SIZE, MAX_LENGTH).to(device)

dataset = TranscriptionDataset()

In [ ]:
train_loop(encoder, decoder, dataset, 1, print_every=1)